<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/C1_feature_generation_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os.path
import json

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive') 
  if not os.path.isfile('SETTINGS.json'):
       # hard coded data directory in drive is used if SETTINGS.json not present 
       config={}
       config['DATA_DIR'] = '/content/gdrive/My Drive/kaggle-c1'
       with open('SETTINGS.json', 'w') as outfile:
         json.dump(config, outfile)

with open('SETTINGS.json') as config_file:
    config = json.load(config_file)

DATA_DIR = config['DATA_DIR']

print('Using DATA_DIR ', DATA_DIR)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Using DATA_DIR  /content/gdrive/My Drive/kaggle-c1


In [0]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from itertools import product
import gc
from tqdm import tqdm_notebook


In [0]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [0]:

DATA_FOLDER = DATA_DIR



In [0]:
all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
# List of columns that we will use to create lags
cols_to_shift = [col for col in all_data.columns.values if re.search('target',col)] 
index_cols = ['shop_id', 'item_id', 'date_block_num']
shift_range = [2, 3, 4, 5,6,12]

# create shifted versions of one column at time and write to disk
# in order to keep memory consumption manageable

for col in cols_to_rename:
    print(col)
    all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
    all_data=all_data[index_cols+[col]]
    all_data=downcast_dtypes(all_data)
    gc.collect()
    for month_shift in tqdm_notebook(shift_range):
        train_shift = all_data[index_cols + [col]].copy()
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x==col else x
        train_shift = train_shift.rename(columns=foo)

        all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
        all_data=downcast_dtypes(all_data)

        del train_shift
        gc.collect()

        # Don't use old data without enough lags in the past
    all_data = all_data[all_data['date_block_num'] >= 12] 
    all_data.to_csv(DATA_FOLDER + '/' + col + '_lagged.csv')
    
    del all_data
    gc.collect()
